In [ ]:
%load_ext kamu
import kamu
con = kamu.connect(engine="spark")

In [ ]:
%%sql
select * from `us.cityofnewyork.data.zipcode-boundaries` limit 3

In [ ]:
%%sql
describe `us.cityofnewyork.data.zipcode-boundaries`

In [ ]:
%%sql -o population_density -q
with with_area as (
    select
        geometry,
        modzcta,
        pop_est as population,
        st_area(st_geomfromgeojson(geometry)) as area
    from `us.cityofnewyork.data.zipcode-boundaries`
)
select
    geometry,
    modzcta,
    population,
    area,
    population / area as density
from with_area

In [ ]:
import os
from mapboxgl.viz import *
from mapboxgl.utils import *
import kamu.utils

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

num_max = population_density.density.max()
stops = [i * num_max / 4 for i in range(4)]

viz = ChoroplethViz(
    kamu.utils.df_to_geojson(population_density),
    style='mapbox://styles/mapbox/dark-v10',
    center=(-74.068266, 40.7019673), zoom=9,
    access_token=token,
    color_property='density',
    color_stops=create_color_stops(stops, colors='YlOrRd'),
    color_function_type='interpolate',
    line_stroke='solid',
    line_width=0.3,
    line_color='rgb(128,0,38)',
    opacity=0.8,)
viz.show()